In [1]:
import networkx as nx
import pandas as pd
import geopandas as gpd
import config as cfg
import numpy as np
import matplotlib.pyplot as plt
from utils import build_trip_count, get_district_names, get_income_data, add_quantiles

In [2]:
all_viajes = pd.read_csv('/Users/caro/Desktop/thesis_project/mobility_data/VIAJES/all_viajes_month_0322.csv') # all trips
income = gpd.read_file('/Users/caro/Desktop/thesis_project/segregation_indices/data/processed/geometries_and_income.geojson') # income data
gdf = gpd.read_file(cfg.ZONIFICACION_DATA / 'distritos/madrid_gdf.geojson') # geodataframe
# gdf = gdf.to_crs(epsg=4326)
district_mapping = pd.read_csv('/Users/caro/Desktop/thesis_project/data_overview/outputs/districts_and_population.csv')

In [3]:
filtered_df = all_viajes.loc[(all_viajes['actividad_origen'] == 'casa')] # home origin trips

In [4]:
filtered_df['avg_distancia'] = filtered_df['viajes_km'] / filtered_df['viajes']

/var/folders/5y/mx06x73d0yb7mfb57y7lb2nh0000gn/T/ipykernel_29191/2907657162.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['avg_distancia'] = filtered_df['viajes_km'] / filtered_df['viajes']


In [7]:
trip_counts = filtered_df.groupby(['origen', 'destino']).size().reset_index(name='trip_count')

# Normalizing trip counts to get weights between 0 and 1
trip_counts['normalized_trip_count'] = (trip_counts['trip_count'] - trip_counts['trip_count'].min()) / (trip_counts['trip_count'].max() - trip_counts['trip_count'].min())

In [8]:
trip_counts

,origen,destino,trip_count,normalized_trip_count
0,2807901,2807901,23450,0.629914
1,2807901,2807902,20538,0.548320
2,2807901,2807903,16117,0.424445
3,2807901,2807904,16966,0.448233
4,2807901,2807905,9214,0.231024
...,...,...,...,...
436,2807921,2807917,1108,0.003895
437,2807921,2807918,1976,0.028216
438,2807921,2807919,2121,0.032279
439,2807921,2807920,19764,0.526633


In [9]:
trip_counts.describe()

,origen,destino,trip_count,normalized_trip_count
count,4.410000e+02,4.410000e+02,441.000000,441.000000
mean,2.807911e+06,2.807911e+06,10973.034014,0.280311
std,6.062178e+00,6.062178e+00,7798.083942,0.218501
min,2.807901e+06,2.807901e+06,969.000000,0.000000
25%,2.807906e+06,2.807906e+06,5110.000000,0.116030
50%,2.807911e+06,2.807911e+06,8315.000000,0.205834
75%,2.807916e+06,2.807916e+06,16414.000000,0.432766
max,2.807921e+06,2.807921e+06,36658.000000,1.000000


In [10]:
trip_counts = get_district_names(trip_counts)
trip_counts = get_income_data(trip_counts, income, 'Gini Index', 'Median income per consumption unit')
trip_counts = add_quantiles(trip_counts, 'Median income per consumption unit', n_quantiles=4)
trip_counts = add_quantiles(trip_counts, 'Gini Index', n_quantiles=4)

In [11]:
trip_counts

,trip_count,normalized_trip_count,origin,destination,Origin Gini Index,Origin Median income per consumption unit,Destination Gini Index,Destination Median income per consumption unit,income decile origin Median income per consumption unit,income decile destination Median income per consumption unit,income decile origin Gini Index,income decile destination Gini Index
0,23450,0.629914,Centro,Centro,39.8,20650,39.8,20650,2,2,3,3
1,20538,0.548320,Centro,Arganzuela,39.8,20650,31.0,26250,2,2,3,0
2,16117,0.424445,Centro,Retiro,39.8,20650,33.1,30450,2,3,3,1
3,16966,0.448233,Centro,Salamanca,39.8,20650,40.2,29750,2,3,3,3
4,9214,0.231024,Centro,Chamartín,39.8,20650,40.2,32550,2,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...
436,1108,0.003895,Barajas,Villaverde,33.2,25550,31.9,15050,2,0,1,1
437,1976,0.028216,Barajas,Villa de Vallecas,33.2,25550,31.1,18550,2,0,1,0
438,2121,0.032279,Barajas,Vicálvaro,33.2,25550,30.4,19250,2,1,1,0
439,19764,0.526633,Barajas,San Blas-Canillejas,33.2,25550,33.8,19250,2,1,1,2


In [24]:
low_to_high_median = trip_counts[(trip_counts['income decile origin Median income per consumption unit'].isin([0, 1])) & 
                                      (trip_counts['income decile destination Median income per consumption unit'].isin([2, 3]))]

high_to_low_median = trip_counts[(trip_counts['income decile origin Median income per consumption unit'].isin([2, 3])) & 
                                      (trip_counts['income decile destination Median income per consumption unit'].isin([0, 1]))]

low_to_low_median = trip_counts[(trip_counts['income decile origin Median income per consumption unit'].isin([0, 1])) &
(trip_counts['income decile destination Median income per consumption unit'].isin([0, 1]))]

high_to_high_median = trip_counts[(trip_counts['income decile origin Median income per consumption unit'].isin([2, 3])) & 
                                trip_counts['income decile destination Median income per consumption unit'].isin([2, 3])]

In [34]:
high_to_low_median

,trip_count,normalized_trip_count,origin,destination,Origin Gini Index,Origin Median income per consumption unit,Destination Gini Index,Destination Median income per consumption unit,income decile origin Median income per consumption unit,income decile destination Median income per consumption unit,income decile origin Gini Index,income decile destination Gini Index
5,7967,0.196083,Centro,Tetuán,39.8,20650,37.6,19250,2,1,3,2
9,15388,0.404018,Centro,Latina,39.8,20650,31.8,17850,2,0,3,0
10,14034,0.366079,Centro,Carabanchel,39.8,20650,33.1,15750,2,0,3,1
11,8841,0.220572,Centro,Usera,39.8,20650,33.4,14350,2,0,3,1
12,5719,0.133094,Centro,Puente de Vallecas,39.8,20650,31.6,14350,2,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...
434,6271,0.148561,Barajas,Ciudad Lineal,33.2,25550,35.8,19950,2,1,1,2
436,1108,0.003895,Barajas,Villaverde,33.2,25550,31.9,15050,2,0,1,1
437,1976,0.028216,Barajas,Villa de Vallecas,33.2,25550,31.1,18550,2,0,1,0
438,2121,0.032279,Barajas,Vicálvaro,33.2,25550,30.4,19250,2,1,1,0


In [25]:
low_to_high_median.shape

(110, 12)

In [26]:
high_to_low_median.shape

(110, 12)

In [27]:
low_to_low_median.shape

(121, 12)

In [28]:
high_to_high_median.shape

(100, 12)

In [37]:
low_to_high_median['trip_count'].sum()

np.int64(1263786)

In [39]:
lh = low_to_high_median['trip_count'].sum() / trip_counts['trip_count'].sum()
hl = high_to_low_median['trip_count'].sum() / trip_counts['trip_count'].sum()
ll = low_to_low_median['trip_count'].sum() / trip_counts['trip_count'].sum()
hh = high_to_high_median['trip_count'].sum() / trip_counts['trip_count'].sum()

In [40]:
lh + hl + ll + hh

np.float64(1.0)